# MQTT Plot

MQTT Plot is a simple python app that plots data sent by MQTT and renders it in a dynamic plot. The following messages are recognized (`MQTT_TOPIC_ROOT` is defined in mqtt_plot/config.py):

* Initialize plot:
  * Topic: `MQTT_TOPIC_ROOT/new`
  * Message (json): `{ "columns": [ "time [s]", "y1", "y2", ... ], "rollover": 200 }`
  * Declaration of the data column names. The number and names of columns is arbitrary. `rollover` specifies the number of points shown. Once more than that many points are received, the plot scrolls to the left. 
* Json data:
  * Topic: `MQTT_TOPIC_ROOT/add`
  * Message (json): `{ "time [s]": 5.39, "y1": 0, "y2": -0.3 }`
  * Data to plot. Typically and (endless) stream.
* Binary data (`from struct import pack`)
  * Topic: `MQTT_TOPIC_ROOT/bin`
  * Message (binary): `pack("!3f", data)`
  * Send binary data (32-bit floats), alternative to send json. The number and order of floats sent must match the columns specified during initialization.

First update the configuration file `$IOT_PROJECTS/mqtt_plot/config.py`. Set `MQTT_BROKER` to match the address of your MQTT broker.

Then start the server from the command line:

```bash
cd $IOT_PROJECTS/robot
python -m mqtt_plot
```

Launch the MQTT data producer (e.g. a microcontroller), first sending to the `new` topic to initialize the plot, followed by data. Open a browser window at the URL indicated by the application (e.g. `http://iot49.local:5006`).

To test the application (without setting up a device to send MQTT data), run the code below to send "sample" data (update `MQTT_BROKER` to match your configuration):

In [9]:
import paho.mqtt.client as mqtt
from time import sleep
import json, math

MQTT_BROKER = "10.39.40.200"
TOPIC_ROOT = "vis"

client = mqtt.Client(clean_session=True, protocol=mqtt.MQTTv311)
client.connect(MQTT_BROKER, port=1883, keepalive=60)

print("sending data ...")
client.publish(f"{TOPIC_ROOT}/new", json.dumps({
    "columns": [ "time [s]", "sin", "cos" ],
    "rollover": 200,
    "args": { "title": "MQTT Plot Demo" },
    # "layout": "scatter_plot"
}))

try:
    t = 0
    while True:
        client.publish(f"{TOPIC_ROOT}/add", 
                       json.dumps({ "time [s]": t, "sin": math.sin(t), "cos": math.cos(t) }))
        t += 0.1
        sleep(0.01)
except KeyboardInterrupt:
    print("so long")

sending data ...
so long


```{figure} figures/mqtt_plot_demo.png
:width: 350px

MQTT Plot Demo
```

## Customization

MQTT Plot makes use of the [Bokeh Visualization Library](https://bokeh.org/). This library offers extensive control and customizations (see [examples](https://docs.bokeh.org/en/latest/docs/gallery.html)).

To customize the output of MQTT Plot, edit a copy of the file `$IOT_PROJECTS/robot/mqtt_plot/layouts/line_plot`). Then in the `new` topic, specify layout to use for plotting, like so (replace `scatter_plot` with the name of your file):

`{ "columns": [ "time [s]", "y1", "y2", ... ], "layout": "scatter_plot" }`